In [19]:
#  Write a program that gets the preference matrix of boys and girls and finds a boy-optimal
#  and a girl-optimal stable marriage.

def is_preferred(girl, boy, boy_preferences, girl_preferences, engaged_pairs,boy_optimal = True):

    if boy_optimal:
      current_engagement = engaged_pairs.get(girl)
      if current_engagement is None:
          return True
      if girl_preferences[girl].index(boy) < girl_preferences[girl].index(current_engagement):
          return True
      return False
    else:
      current_engagement = engaged_pairs[boy]

      if current_engagement is None:
          return True
      if boy_preferences[boy].index(girl) < boy_preferences[boy].index(current_engagement):
          return True
      return False

def stable_marriage(boy_preferences, girl_preferences):
    n = len(boy_preferences)
    engaged_boys = set()
    engaged_pairs = {}

    while len(engaged_boys) < n:
        # Find the first unengaged boy
        boy = next(boy for boy in range(n) if boy not in engaged_boys)
        got_offer = False
        for girl in boy_preferences[boy]:
            if girl not in engaged_pairs.keys():
                # The girl is unengaged, pair the boy and the girl
                got_offer = True
                engaged_pairs[girl] = boy
                engaged_boys.add(boy)
                break
            else:
                current_boy = engaged_pairs[girl]
                if is_preferred(girl, boy, boy_preferences, girl_preferences, engaged_pairs):
                    # The girl prefers the current boy over the previous engagement
                    got_offer = True
                    engaged_pairs[girl] = boy
                    engaged_boys.add(boy)
                    engaged_boys.remove(current_boy)
                    break
        if not got_offer:
          engaged_boys.add(boy)
    boy_optimal_marriage = engaged_pairs

    n = len(girl_preferences)
    engaged_pairs = {}
    engaged_girls = set()

    while len(engaged_girls) < n:
        girl = next(girl for girl in range(n) if girl not in engaged_girls)
        got_offer = False
        for boy in girl_preferences[girl]:
            if boy not in engaged_pairs.keys():
                engaged_pairs[boy] = girl
                engaged_girls.add(girl)
                got_offer = True
                break
            else:
                current_girl = engaged_pairs[boy]
                if is_preferred(girl, boy, boy_preferences, girl_preferences, engaged_pairs,False):
                    engaged_pairs[boy] = girl
                    engaged_girls.add(girl)
                    engaged_girls.remove(current_girl)
                    got_offer = True
                    break
        if not got_offer:
          engaged_girls.add(girl)

    girl_optimal_marriage = engaged_pairs
    return boy_optimal_marriage, girl_optimal_marriage

# Example 1:
boy_preferences = [
    [0, 1, 2],  # Preferences of Boy 0
    [2, 0, 1],  # Preferences of Boy 1
    [0, 1, 2],   # Preferences of Boy 2
    [0, 1, 2]   # Preferences of Boy 3
]

girl_preferences = [
    [0, 1, 2,3],  # Preferences of Girl 0
    [1, 0, 2,3],  # Preferences of Girl 1
    [0, 1, 2,3]   # Preferences of Girl 2
]

boy_optimal, girl_optimal = stable_marriage(boy_preferences, girl_preferences)

print("Boy-Optimal Marriage:")
for girl, boy in boy_optimal.items():
    print(f"Boy {boy} - Girl {girl}")

print("\nGirl-Optimal Marriage:")
for boy, girl in girl_optimal.items():
   print(f"Girl {girl} - Boy {boy}")
print('-------------------------------------------------------')
# Example 2:
boy_preferences = [
    [0, 2, 3,1],  # Preferences of Boy 0
    [1, 3, 2,0],  # Preferences of Boy 1
    [3, 0, 2,1],   # Preferences of Boy 2
    [1, 2, 3,0]   # Preferences of Boy 3
]

girl_preferences = [
    [2, 3, 0,1],  # Preferences of Girl 0
    [1, 2, 0,3],  # Preferences of Girl 1
    [2, 0, 3,1],   # Preferences of Gril 2
    [1, 0, 2,3]   # Preferences of Girl 3
]

boy_optimal, girl_optimal = stable_marriage(boy_preferences, girl_preferences)

print("Boy-Optimal Marriage:")
for girl, boy in boy_optimal.items():
    print(f"Boy {boy} - Girl {girl}")

print("\nGirl-Optimal Marriage:")
for boy, girl in girl_optimal.items():
   print(f"Girl {girl} - Boy {boy}")


Boy-Optimal Marriage:
Boy 0 - Girl 0
Boy 1 - Girl 2
Boy 2 - Girl 1

Girl-Optimal Marriage:
Girl 0 - Boy 0
Girl 2 - Boy 1
Girl 1 - Boy 2
-------------------------------------------------------
Boy-Optimal Marriage:
Boy 0 - Girl 0
Boy 1 - Girl 1
Boy 2 - Girl 3
Boy 3 - Girl 2

Girl-Optimal Marriage:
Girl 3 - Boy 2
Girl 1 - Boy 1
Girl 2 - Boy 0
Girl 0 - Boy 3


In [5]:
# Implement top trading cycle algorithm, which gets the preferences of the agents and
# outputs a stable permutation.

import networkx as nx

def get_students(G, cycle, students):

   if cycle in students:
      cycle = list(G.edges(cycle))[0][1]

   starting_room = cycle
   students_in_cycle = set()
   curr_node = list(G.edges(starting_room))[0][1]

   while curr_node not in students_in_cycle:
      students_in_cycle.add(curr_node)
      curr_node = list(G.edges(curr_node))[0][1]
      curr_node = list(G.edges(curr_node))[0][1]
   return students_in_cycle


def find_cycle(G):
   visited_nodes = set()
   nudes = list(G.nodes())
   v = nudes[0]
   while v not in visited_nodes:
      visited_nodes.add(v)
      v = list(G.edges(v))[0][1]
   return v


def top_trading_cycles(students, rooms, students_preferences, init_alloc):


   nodes = set(students) | set(rooms)
   students = set(students)
   G = nx.DiGraph()
   G.add_nodes_from(nodes)
   preference_index = dict((x,0) for x in students)
   preferred_room_fun = lambda x: students_preferences[x][preference_index[x]]


   for a in students:
      G.add_edge(a, preferred_room_fun(a))
   for h in rooms:
      G.add_edge(h, init_alloc[h])
   answer = dict()


   while len(list(G.nodes())) > 0:
      cycle = find_cycle(G)
      students_in_cycle = get_students(G, cycle, students)


      for a in students_in_cycle:
         h = list(G.edges(a))[0][1]
         answer[a] = h
         G.remove_node(a)
         G.remove_node(h)


      for a in students:
         if a in list(G.nodes()) and G.out_degree(a) == 0:
            while preferred_room_fun(a) not in list(G.nodes()):
               preference_index[a] += 1
            G.add_edge(a, preferred_room_fun(a))


   return answer

In [7]:
students = {'a','b','c','d','e','f'}
rooms = {1,2,3,4,5,6}
students_preferences = {
  'a': [6,1,3,4,5,2],
  'b': [3,2,1,4,5,6],
  'c': [5,1,6,4,2,3],
  'd': [2,1,3,4,5,6],
  'e': [1,2,3,4,5,6],
  'f': [5,2,3,4,6,1],
}

init_alloc = {1:'a', 2:'b', 3:'c', 4:'d',5:'e',6:'f'}
print('expected answer:')
print({'a': 6, 'b':3, 'c':4, 'd':2, 'e':1, 'f':5})
print('retured answer:')
print(top_trading_cycles(students, rooms, students_preferences, init_alloc))

expected answer:
{'a': 6, 'b': 3, 'c': 4, 'd': 2, 'e': 1, 'f': 5}
retured answer:
{'e': 1, 'a': 6, 'f': 5, 'c': 4, 'd': 2, 'b': 3}
